<a href="https://colab.research.google.com/github/kevin801221/LLMs_Amazing_courses_Langchain_LlamaIndex/blob/main/building_llm_app_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />
<!--- @wandbcode{python-report-api} -->

In this minimally annotated notebook, I'll build a simple LLM-powered app with Gradio, W&B, and Chroma (and a tiny bit of LangChain). The purpose of this notebook is to show:
- creating a simple Hugging Face transformers pipeline for image-to-text
- how to build a simple Chatbot interface with gradio
- W&B Prompts feature for handling LLMs
- how to use a vector database with Chroma
- creating a simple prompt template with LangChain



# 🔩 Setup

We will use:
- [wandb](https://wandb.ai/site) for logging and tracking
- [transformers](https://huggingface.co/docs/transformers/index) for loading in our model and pipeline
- [gradio](https://www.gradio.app/) for creating the app
- [chromadb](https://docs.trychroma.com/) for storing model generations
- [openai](https://pypi.org/project/openai/) for the embedding function used in our Chroma database
- [tiktoken](https://github.com/openai/tiktoken) for our OpenAI Embedding
- [langchain](https://www.langchain.com/) for a simple prompt template
- [accelerate](https://huggingface.co/docs/accelerate/index) for quantized loading for our HF pipeline
- [bitsandbytes](https://huggingface.co/docs/accelerate/usage_guides/quantization) for quantized loading for our HF pipeline

In [ ]:
!pip install wandb -qqq
!pip install git+https://github.com/huggingface/transformers -qqq
!pip install --upgrade gradio -qqq
!pip install chromadb -qqq
!pip install openai -qqq
!pip install tiktoken -qqq
!pip install langchain -qqq
!pip install accelerate -qqq
!pip install bitsandbytes -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 46.6 MB/s eta 0:00:00
     ━━━━

In [ ]:
import os
import requests
import numpy as np
import torch
import datetime

# For loading in the tiny-LLaVA-v1-hf model in a transformers pipeline.
import transformers
from transformers import pipeline
from transformers import BitsAndBytesConfig

# For converting input images to PIL images.
from PIL import Image

# For creating the gradio app.
import gradio as gr

# For creating a simple prompt (open to extension) to our model.
from langchain.prompts import PromptTemplate

# Our vector database of choice: Chroma!
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.openai import OpenAIEmbeddings

import chromadb
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.data_loaders import ImageLoader

# For loading in our OpenAI API key.
from google.colab import userdata

# For logging.
import wandb
from wandb.sdk.data_types.trace_tree import Trace
wandb.login()

# Required for us to load in our pipeline for TinyLLaVA.
assert transformers.__version__ >= "4.35.3"

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# 🧬 Chroma: Vector Database with OpenAI

In [ ]:
# Use OpenAI's embeddings for our Chroma collection.
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=userdata.get("OPENAI_API_KEY"),
)
collection = Chroma("conversation_memory", embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


# 🧪 Pipeline

In [ ]:
# Ref: https://huggingface.co/bczhou/tiny-llava-v1-hf
model_id = "bczhou/tiny-llava-v1-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

pipe = pipeline(
    "image-to-text",
    model=model_id,
    device_map="auto",
    use_fast=True,
    model_kwargs={"quantization_config": bnb_config}
)

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/661M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

# 🚚 Building the App

In [ ]:
#@title Optional, pass in a path or URI to an image!
user_avatar_image_path = "" # @param {type:"string"}
chatbot_avatar_image_path = "" # @param {type:"string"}

try:
  assert user_avatar_image_path
except:
  img_data = requests.get("https://imgur.com/QehpHeV.png").content
  with open('user_avatar.png', 'wb') as handler:
      handler.write(img_data)
  user_avatar_image_path = "user_avatar.png"

try:
  assert chatbot_avatar_image_path
except:
  img_data = requests.get("https://imgur.com/ki4hPhZ.png").content
  with open('chatbot_avatar.png', 'wb') as handler:
      handler.write(img_data)
  chatbot_avatar_image_path = "chatbot_avatar.png"

In [ ]:
print(user_avatar_image_path, chatbot_avatar_image_path)

user_avatar.png chatbot_avatar.png


In [ ]:
# Let's get a sample image to use. You can download it and pass it into the app!
# The prompt is: What's unusual about this image?
img_data = requests.get("https://imgur.com/Ca6gjuf.png").content
with open('sample_image.png', 'wb') as handler:
    handler.write(img_data)

In [ ]:
max_new_tokens = 200

# Path for storing images.
IMG_ROOT_PATH = "data/"
os.makedirs(IMG_ROOT_PATH, exist_ok=True)

# Define the function with (message, history) + additional_inputs -> str.
def generate_output(message: str, history: list, img: np.ndarray) -> str:
    """Generates an output given a message and image."""
    status = "success"

    # Get detailed description of the image for Chroma.
    query = "Please provide a detailed description of the image."
    prompt = PromptTemplate.from_template(
        "USER: <image>\n" +
        "{query}" +
        "\n" +
        "ASSISTANT: "
    )

    start_time_ms = datetime.datetime.now().timestamp() * 1000
    try:
        outputs = pipe(Image.fromarray(img), prompt=prompt.format(query=query), generate_kwargs={"max_new_tokens": max_new_tokens})
        img_desc = outputs[0]["generated_text"].split("ASSISTANT:")[-1]
        status_message = (None,)
    except Exception as e:
        status = "error"
        status_message = str(e)
        img_desc = ""
    end_time_ms = round(datetime.datetime.now().timestamp() * 1000)

    # Create a span in wandb.
    root_span = Trace(
        name="img_desc_span",
        kind="llm",  # kind can be "llm", "chain", "agent" or "tool"
        status_code=status,
        status_message=status_message,
        metadata={
            "max_new_tokens": max_new_tokens,
            "model_name": model_id,
        },
        start_time_ms=start_time_ms,
        end_time_ms=end_time_ms,
        inputs={"system_prompt": prompt.format(query=query), "query": query},
        outputs={"response": img_desc},
    )

    # Log the span to wandb.
    root_span.log(name="img_desc_trace")

    # Visual Question-Answering!
    prompt = PromptTemplate.from_template(
        "Context: {context}\n\n"
        "USER: <image>\n" +
        "{message}" +
        "\n" +
        "ASSISTANT: "
    )
    context = collection.similarity_search(query=message, k=2)
    context = "\n".join([doc.page_content for doc in context])

    # Forward pass through the model with given prompt template.
    start_time_ms = datetime.datetime.now().timestamp() * 1000
    try:
        outputs = pipe(
            Image.fromarray(img),
            prompt=prompt.format(
                context=context,
                message=message
            ),
            generate_kwargs={"max_new_tokens": max_new_tokens}
        )
        response = outputs[0]["generated_text"].split("ASSISTANT:")[-1]
        status_message = (None,)
    except Exception as e:
      status = "error"
      status_message = str(e)
      response = ""
    end_time_ms = round(datetime.datetime.now().timestamp() * 1000)

    # Create a span in wandb.
    root_span = Trace(
        name="response_span",
        kind="llm",  # kind can be "llm", "chain", "agent" or "tool"
        status_code=status,
        status_message=status_message,
        metadata={
            "max_new_tokens": max_new_tokens,
            "model_name": model_id,
        },
        start_time_ms=start_time_ms,
        end_time_ms=end_time_ms,
        inputs={
            "system_prompt": prompt.format(
                context=context,
                message=message
            ),
            "query": message
        },
        outputs={"response": response},
    )

    # Log the span to wandb.
    root_span.log(name="response_trace")

    # Add (img_desc, message, response) 3-tuple to Chroma collection.
    text = f"Image Description: {img_desc}\nUSER: {message}\nASSISTANT: {response}\n"
    collection.add_texts(texts=[text])

    # Return model output.
    return img_desc + "\n\n" + response

In [ ]:
# wandb.finish()

In [ ]:
wandb.init(project="building_llm_app")

wandb: Currently logged in as: vincenttu. Use `wandb login --relogin` to force relogin


In [ ]:
# Define the ChatInterface, customize, and launch!
gr.ChatInterface(
    generate_output,
    chatbot=gr.Chatbot(
        label="Chat with me!",
        show_label=True,
        container=False,
        scale=5,
        height=300,
        show_share_button=True,
        show_copy_button=True,
        avatar_images=(user_avatar_image_path, chatbot_avatar_image_path),
        likeable=False,
        layout="bubble",
        bubble_full_width=False
      ),
    textbox=gr.Textbox(
        lines=1,
        max_lines=5,
        placeholder="Message ...",
        container=False,
        scale=7,
        info="Input your textual response in the text field and your image below!"
    ),
    additional_inputs="image",
    additional_inputs_accordion=gr.Accordion(
        open=True,
    ),
    title="Language-Image Question Answering with bczhou/TinyLLaVA-v1-hf!",
    description="""
    This simple gradio app internally uses a Large Language-Vision Model (LLVM) and the Chroma vector database for memory.
    Note: this minimal app requires both an image and a text-based query before the chatbot system can respond.
    """,
    theme="soft",
    submit_btn="Submit ▶",
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch(debug=True, share=True)

/usr/local/lib/python3.10/dist-packages/gradio/components/base.py:182: UserWarning: show_label has no effect when container is False.
  warnings.warn("show_label has no effect when container is False.")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://0f9515b1c63de2991d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0f9515b1c63de2991d.gradio.live


In [ ]:
wandb.finish()

# 🙏 References

**Setup**
- https://wandb.ai/site
- https://huggingface.co/docs/transformers/index
- https://www.gradio.app/
- https://docs.trychroma.com/
- https://pypi.org/project/openai/
- https://www.langchain.com/
- https://huggingface.co/docs/accelerate/index
- https://huggingface.co/docs/accelerate/usage_guides/quantization

**Chroma: Vector Database with OpenAI**
- https://github.com/chroma-core/chroma
- https://docs.trychroma.com/

**Optional, Multi-Modal Vector Database with Chroma**
- https://docs.trychroma.com/multi-modal?lang=py
- https://github.com/chroma-core/chroma/blob/a370684dd032eaf52ad9619c4811449a52cc1e2c/chromadb/utils/embedding_functions.py#L666
- https://github.com/chroma-core/chroma/blob/a370684dd032eaf52ad9619c4811449a52cc1e2c/chromadb/utils/data_loaders.py#L9
- https://github.com/chroma-core/chroma/blob/a370684dd032eaf52ad9619c4811449a52cc1e2c/chromadb/api/client.py#L115
- https://github.com/chroma-core/chroma/blob/a370684dd032eaf52ad9619c4811449a52cc1e2c/chromadb/api/client.py#L188


**Pipeline**
- https://huggingface.co/bczhou/tiny-llava-v1-hf
